# Object Following - Live Demo

このノートブックでは、JetBotでオブジェクトを追跡する方法を示します。 事前にトレーニングされたニューラルネットワークを使用してオブジェクト検出します。このオブジェクト検出モデルは一般的な90種類のオブジェクトの画像を分類した[COCOデータセット](http://cocodataset.org)を使ってトレーニングされています。COCOデータセットには

* 人（インデックス0）
* カップ（インデックス47）

その他多数あります（クラスインデックスの完全なリストについては、[このファイル](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt)で確認できます）。 モデルは[TensorFlowオブジェクト検出API](https://github.com/tensorflow/models/tree/master/research/object_detection)から供給され、カスタムタスクのオブジェクト検出器をトレーニングするためのユーティリティも提供します。 TensorFlowのモデルで学習が終わった後、Jetson NanoのNVIDIA TensorRTを使用してモデルを最適化します。

これにより、ネットワークは非常に高速になり、Jetson Nanoでリアルタイムに実行できるようになります。 ただし、このノートブックではCOCOデータセットからのトレーニングや他の最適化に関する手順は実行しません。

とにかく、始めましょう。最初に、事前トレーニング済みのSSDエンジンを使用する``ObjectDetector``クラスをインポートします。

### 単一のカメラ画像で検出する

In [ ]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

内部的には、``ObjectDetector``クラスはTensorRT Python APIを使用して、提供するエンジンを実行します。 また、ニューラルネットワークへの入力の前処理や、検出されたオブジェクトの解析も行います。 現時点では、``jetbot.ssd_tensorrt``パッケージを使用して作成されたエンジンでのみ機能します。 このパッケージには、モデルをTensorFlowオブジェクト検出APIから最適化されたTensorRTエンジンに変換するためのユーティリティが含まれています。

次に、カメラを初期化しましょう。 検出器は300x300ピクセルの入力を受け取るため、カメラを作成するときにこれを設定します。

> 内部的には、CameraクラスはGStreamerを使用してJetson Nanoの画像信号プロセッサ（ISP）を利用しています。 これは超高速であり、CPUからサイズ変更の計算の多くをオフロードします。

In [ ]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

次に、カメラ入力を使用してネットワークを実行してみましょう。 デフォルトでは ``ObjectDetector``クラスはカメラが生成する``bgr8``フォーマットを期待しています。 しかし、別のフォーマットを入力に使う場合は、デフォルトの前処理関数をオーバーライドして変更できます。

In [ ]:
detections = model(camera.value)

print(detections)

カメラの視野にCOCOオブジェクトがある場合、それらは``detections``変数に格納されるはずです。

### テキスト領域に検出を表示する

次のコードを使用して、検出されたオブジェクトを出力します。

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()

detections_widget.value = str(detections)

display(detections_widget)

各画像で検出された各オブジェクトのラベル、信頼度、境界ボックスが表示されます。 この例では、1つの画像（カメラ）しかありません。

最初の画像で検出された最初のオブジェクトのみを表示するには、次のように呼び出すことができます

> オブジェクトが検出されない場合、エラーが発生する可能性があります

In [ ]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

### 中心物体を追跡するようにロボットを制御する

次に、ロボットに指定されたクラスのオブジェクトを追跡させます。 これを行うには、次のようにします

1.  指定したクラスに一致するオブジェクトを検出します。
2.  カメラの視野の中心に最も近いオブジェクトを選択します。これは`ターゲット`オブジェクトです。
3.  ロボットをターゲットオブジェクトに向けます。
4.  障害物によってブロックされている場合は、左折します。

また、ターゲットオブジェクトのラベル、ロボットの速度を制御するために使用するいくつかのウィジェットを作成します。
`turn gain`は、ターゲットオブジェクトとロボットの視野の中心との間の距離に基づいてロボットが回転する速度を制御します。

まず、衝突検出モデルをロードします。 事前トレーニング済みモデルは便宜上このディレクトリに保存されます。
衝突回避の例に従っている場合は、実際の環境でうまく動作するモデルを使用することをお勧めします。


In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('../collision_avoidance/best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

それでは、ロボットを初期化して、モーターを制御できるようにしましょう。

In [ ]:
from jetbot import Robot

robot = Robot()

最後に、すべてのコントロールウィジェットを表示し、カメラの更新とモデルの実行を連動させます。

In [ ]:
from jetbot import bgr8_to_jpeg

blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    image = change['new']
    
    # execute collision model to determine if blocked
    collision_output = collision_model(preprocess(image)).detach().cpu()
    prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    blocked_widget.value = prob_blocked
    
    # turn left if blocked
    if prob_blocked > 0.5:
        robot.left(0.3)
        image_widget.value = bgr8_to_jpeg(image)
        return
        
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    
        
    # otherwise go forward if no target detected
    if det is None:
        robot.forward(float(speed_widget.value))
        
    # otherwsie steer towards target
    else:
        # move robot forward and steer proportional target's x-distance from center
        center = detection_center(det)
        robot.set_motors(
            float(speed_widget.value + turn_gain_widget.value * center[0]),
            float(speed_widget.value - turn_gain_widget.value * center[0])
        )
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

以下のブロックを呼び出して、カメラフレームの更新毎に実行するようにします。

In [ ]:
camera.unobserve_all()
camera.observe(execute, names='value')

驚くばかり！ ロボットがブロックされていない場合は、検出されたオブジェクトの周りに描かれたボックスが青色で表示されます。 ロボットが追跡するターゲットオブジェクトが緑色で表示されます。

ロボットはそれが検出されたときにターゲットに向かって操縦する必要があります。 オブジェクトによってブロックされている場合は、左に曲がります。

以下のコードブロックを呼び出して、カメラから処理を手動で切断し、ロボットを停止できます。

In [ ]:
import time

camera.unobserve_all()
time.sleep(1.0)
robot.stop()